In [35]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras 
from tensorflow.keras import layers
import os
from PIL import Image
from tensorflow.keras.utils import image_dataset_from_directory

In [31]:
'''Tis will load the data with tf'''
folder_path=r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\logo findder"

train = image_dataset_from_directory(folder_path,
    seed=42,
    validation_split=0.2,
    subset='training',
    image_size=(224,224),
)
#https://www.tensorflow.org/tutorials/images/transfer_learning

Found 445 files belonging to 15 classes.
Using 356 files for training.


In [30]:
# Print filenames of images in the dataset directory
for root, dirs, files in os.walk(r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\logo findder"):
    for file in files:
        print(os.path.join(file))

Image_1.jpg
Image_10.png
Image_11.jpg
Image_12.jpg
Image_13.png
Image_14.JPG
Image_15.jpg
Image_16.jpg
Image_17.jpg
Image_18.png
Image_19.jpg
Image_2.jpg
Image_20.png
Image_21.png
Image_22.jpg
Image_23.jpg
Image_24.jpg
Image_25.jpg
Image_26.jpg
Image_27.jpg
Image_28.png
Image_29.jpg
Image_3.png
Image_30.jpg
Image_4.jpg
Image_5.png
Image_6.jpg
Image_7.jpg
Image_8.jpg
Image_9.jpg
Image_1.png
Image_10.png
Image_11.png
Image_12.png
Image_13.jpg
Image_14.jpg
Image_15.jpg
Image_16.JPG
Image_17.jpg
Image_18.png
Image_19.jpg
Image_2.png
Image_20.jpg
Image_21.jpg
Image_22.jpg
Image_23.png
Image_24.jpg
Image_25.jpg
Image_26.jpg
Image_27.jpg
Image_28.jpg
Image_29.jpg
Image_3.jpg
Image_30.jpg
Image_4.jpg
Image_5.jpg
Image_6.png
Image_7.png
Image_8.jpg
Image_9.jpg
Image_1.jpg
Image_10.jpg
Image_11.jpg
Image_12.jpg
Image_13.png
Image_14.png
Image_15.jpg
Image_16.jpg
Image_17.png
Image_18.jpg
Image_19.jpg
Image_2.jpg
Image_20.jpg
Image_21.jpg
Image_22.gif
Image_23.jpg
Image_24.jpg
Image_25.gif
Image_

In [39]:
val =image_dataset_from_directory(folder_path, 
    seed=42,
    validation_split=0.2,
    subset='validation',
    image_size=(150,150),
)
# Printing class names
class_names = train.class_names
print("Class names:", class_names)

Found 445 files belonging to 15 classes.
Using 89 files for validation.
Class names: ['Original Aston Martin logo', 'Original Audi  logo', 'Original Bentley logo', 'Original Bugatti logo', 'Original Ferrari logo', 'Original Fisker Automotive logo', 'Original Ford  logo', 'Original Koenigsegg logo', 'Original Lamborghini logo', 'Original Lotus Cars logo', 'Original Maserati logo', 'Original McLaren logo', 'Original Mercedes-AMG  logo', 'Original Pagani logo', 'Original Porsche logo']


In [27]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True, 
                                                     weights='imagenet')
pretrained_model.trainable = False
pretrained_model.compile(optimizer='adam', 
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                         metrics=['accuracy'])
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [28]:
result = pretrained_model.evaluate(train)
print(dict(zip(pretrained_model.metrics_names, result)))

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_13347]

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [36]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

resModel = Sequential()
resModel.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
resModel.add(Dense(1, activation='sigmoid'))
# Unfreeze some layers for fine-tuning
for layer in resModel.layers[0].layers[-50:]:
    layer.trainable = True

In [38]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


optimizer = Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

resModel.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 2
history = resModel.fit(train, epochs=epochs, validation_data=val, callbacks=[reduce_lr])

Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_65911]